In [120]:
from importlib import reload # python 2.7 does not require this
from yahoofinancials import YahooFinancials
import numpy as np
import json as js

In [121]:
class cookFinancials(YahooFinancials):
    ticker = ''
    bshData = []
    def __init__(self, ticker):
        if isinstance(ticker, str):
            self.ticker = ticker.upper()
        else:
            self.ticker = [t.upper() for t in ticker]
        self._cache = {}
        
    def get_balanceSheetHistory(self):
        self.bshData = self.get_financial_stmts('annual', 'balance')['balanceSheetHistory']
        return self.bshData
    
    def get_BV(self, numofYears=20):
        bv = []
        if not(self.bshData):
            self.get_balanceSheetHistory()
        for i in range(min(np.size(self.bshData[self.ticker]), numofYears)):
            date_key = list(self.bshData[self.ticker][i].keys())[0]
            if not(self.bshData[self.ticker][i][date_key]):    
                break
            bv.append(self.bshData[self.ticker][i][date_key]['totalStockholderEquity'])
        return bv   
    #use mean of each year    
    def get_BV_GR_median(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        #print(gr)
        return np.size(bv)-1, np.median(gr)
    
    def get_BV_GR_max(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        #print(gr)
        return np.size(bv)-1, np.max(gr)
    
    def get_BV_GR_mean(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        #print(gr)
        return np.size(bv)-1, np.mean(gr)
    
    def get_suggest_price(self, cEPS, growth, years, rRate, PE, safty):
        if not(cEPS) or not(growth) or not(PE):
            return 'NA'
        fEPS = cEPS*(1+growth)**years
        fPrice = fEPS*PE;
        stickerPrice = fPrice/(1+rRate)**years
        return stickerPrice, stickerPrice*safty
    
    def get_PE(self):
        #print(self._stock_summary_data('trailingPE'))
        #print(self._stock_summary_data('forwardPE'))
        if not(self._stock_summary_data('trailingPE')):
            return self._stock_summary_data('forwardPE')
        if not(self._stock_summary_data('forwardPE')):
            return self._stock_summary_data('trailingPE')
        return (self._stock_summary_data('trailingPE')+self._stock_summary_data('forwardPE'))/2
    
    def get_decision(self,suggestPrice, stockprice):
        print('suggested price:', suggestPrice)
        print('stock price:', stockprice)
        if isinstance(suggestPrice, str):
            return 'no recommendation due to error'
        elif suggestPrice>stockprice:
            return 'strong buy' 
        else:
            return 'do not buy'
            
            
        

In [132]:
class batch_process:
    def __init__(self, tickers, section):
        self.tickers = tickers
        self.jsfile = section
        with open(section, "w") as f:
            s = {"data":[]}
            js.dump(s, f, indent = 4)
    def batch_run(self):       
        for i in range(np.size(self.tickers)):
            try:
                print(self.tickers[i])
                x = cookFinancials(self.tickers[i])
                bv = x.get_BV(20)
                bv.insert(0, x.get_book_value())
                #print(bv)
                bvgr = x.get_BV_GR_median(bv)
                print(bvgr)
                growth = bvgr[1]
                cEPS = x.get_earnings_per_share()
                years = 3;
                rRate = 0.2;
                safty = 0.5
                PE = x.get_PE()
                price=x.get_suggest_price(cEPS, growth, years, rRate, PE, safty)
                #print(price)
                print(x.get_decision(price[1],x.get_current_price()))
                s = {
                    self.tickers[i]:{
                        "suggested price":price[1],
                        "stock price":x.get_current_price(),
                        "decision":(x.get_decision(price[1],x.get_current_price()))
                    }
                }
                with open(self.jsfile, "r") as f:
                    data = js.load(f)
                    cont = data['data']
                    cont.append(s)
                with open(self.jsfile, "w") as f:
                    js.dump(data, f, indent=4) 
                print('=====================================')
            except Exception:
                pass

In [123]:
x = cookFinancials('MRVI')

In [124]:
bv = x.get_BV(20)
bv

[141396000, 144405000]

In [125]:
not(list(x.get_financial_stmts('quarterly', 'balance').values())[0]['MRVI'])

True

In [137]:
try:
    x.get_book_value()
except Exception:
    pass

In [746]:
bvgr = x.get_BV_GR_median(bv)
print(bvgr)

(4, 0.3381376283195572)


In [747]:
growth = bvgr[1]
cEPS = x.get_earnings_per_share()
years = 3;
rRate = 0.2;
safty = 0.5
PE = x.get_PE()

In [748]:
print(x.get_earnings_per_share())


2.6480002393131707


In [749]:
x.get_pe_ratio()
x.get_PE()

66.858

In [750]:
print(cEPS)
print(growth)
print(years)
print(rRate)
print(PE)
print(safty)

2.6480002393131707
0.3381376283195572
3
0.2
66.858
0.5


In [751]:
price=x.get_suggest_price(cEPS, growth, years, rRate, PE, safty)
print(price)

(245.48785567341707, 122.74392783670854)


In [752]:
x.get_decision(price[1],x.get_current_price())

suggested price: 122.74392783670854
stock price: 177.04


'do not buy'

In [56]:
from get_all_tickers import get_tickers as gt
from get_all_tickers.get_tickers import Region

In [7]:
tickers = gt.get_tickers()



In [59]:
filtered_by_sector = gt.get_tickers_filtered(mktcap_min=1e3, sectors=gt.SectorConstants.HEALTH_CARE)

In [138]:
filtered_by_sector.index('MRVI')

238

In [139]:
y = batch_process(filtered_by_sector[238:], 'health1.json')

In [ ]:
y.batch_run()


MRVI
MASI
(4, 0.251896314842176)
suggested price: 149.0236605221484
stock price: 261.47
do not buy
suggested price: 149.0236605221484
stock price: 261.47
MEDP
(4, 0.13055547981522453)
suggested price: 49.64212837809528
stock price: 132.89
do not buy
suggested price: 49.64212837809528
stock price: 132.89
MMSI
(4, 0.18799577079680552)
suggested price: A
stock price: 55.55
no recommendation due to error
suggested price: A
stock price: 55.55
MRSN
(4, 2.2464035503769106)
suggested price: A
stock price: 22.99
no recommendation due to error
suggested price: A
stock price: 22.99
MESO
(4, 0.01844483762541113)
suggested price: A
stock price: 15.74
no recommendation due to error
suggested price: A
stock price: 15.74
MRTX
(4, 0.6688382880753934)
suggested price: A
stock price: 231.51
no recommendation due to error
suggested price: A
stock price: 231.51
MRNA
